In [2]:
import numpy as np
import pyphi
from itertools import chain,combinations
from scipy import stats
import pandas as pd

In [76]:
class phi():
    def __init__(self,tpm,states=None):
        self.tpm = tpm
        self.num_nodes = int(np.log2(len(tpm)))
        if states is None:
            self.states = np.zeros((self.num_nodes))
        else:
            self.states = states
        
    # Converts a list of states to a row number, so that we can index the tpm
    # input : list of states, Ex. [0,1,0,0]
    # output : row number based on little endian notation, so index zero in list is least
    #          significant bit and index n-1 is most significant bit. Ex. [0,1,0,0] -> 2
    def state_to_index(self,states):
        decimal = 0
        for i,state in enumerate(states):
            decimal += (2**i) * (state)
        return decimal
    
    # Converts a row number from the tpm into a list of states
    # input : row number, total number of states. Ex. (2, 4)
    # output : list of states that row number represents. Ex. (2, 4) -> [0,1,0,0]
    #          We include the number of states so that we know how many zeros to append at the end.
    def index_to_state(self,index,num_states):
        binary = bin(index)[2:]
        states = []
        for state in binary: # convert row to binary
            states.append(int(state))
        
        states = np.flip(states) # flip to make little endian
       
        if (len(states) < num_states): 
            states = np.concatenate((states,np.zeros((num_states-len(states)),dtype=int)))
            
        return states
    
    # Because they're conditional probability distributions, the rows need to sum to one
    def normalize_rows(self,tpm):
        return tpm/np.sum(tpm,1)[:, np.newaxis]
    
    # return the powerset of a list of nodes
    def powerset(self,iterable):
        "powerset([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3)"
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(1,len(s)))
    
    # return the powerset of a list of nodes including full set
    def powerset_all(self,iterable):
        "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(0,len(s)+1))
    
    # returns the tensor product of two tpm (effect)
    def tensor_product(self,t1,t2):
        tensor_product = np.zeros((t1.shape[0],t1.shape[1]*t2.shape[1]))
        column = 0
        for c2 in t2.T:
            for c1 in t1.T:
                tensor_product[:,column] = c1*c2
                column+=1
                
        return tensor_product
    
    # This tensor product combines two effect repertoires t1 and t2, given a list of their nodes also
    # This ensures that the nodes get ordered correctly even if the purview nodes of t1 are 'B' and the
    # purview nodes of t2 are 'A,C'.
    def tensor_product_ordered(self,t1,t1_nodes,t2,t2_nodes):
        tensor_product = np.zeros((t1.shape[0],t1.shape[1]*t2.shape[1]))
        
        # when we perform the tensor product we want to put the effect nodes in the correct order in the
        # new tpm, but the purview might not be the full system of nodes, so we create a sorted list
        # and index each purview node in the list
        sorted_nodes = np.sort(t1_nodes + t2_nodes)
        ordered_indexes = np.zeros((self.num_nodes),dtype=int)
        order = 0
        
        for node in sorted_nodes:
            ordered_indexes[node] = order
            order+=1

        columns = tensor_product.shape[1]
        
        # Fill the columns of the tpm one at a time
        for column in range(columns):
            # find the states of t1's nodes given the column we're currently filling
            t1_state = self.index_to_state(column,int(np.log2(columns)))[ordered_indexes[t1_nodes]]
            # do the same for t2's nodes
            t2_state = self.index_to_state(column,int(np.log2(columns)))[ordered_indexes[t2_nodes]]
            # Then fill the column with the correct column from t1 multiplied by the correct column from
            # t2
            tensor_product[:,column] = t1[:,self.state_to_index(t1_state)] * t2[:,self.state_to_index(t2_state)]
            
        return tensor_product
        
    """
    effect_repertoire :
        input : mechanism (t nodes), purview (t+1 nodes)
        algorithm : from original tpm find the tpm P(purview | mechanism)
        output : repertoire for those nodes
    """    
    def effect_repertoire(self,mechanism,purview):
        # The effect repertoire captures the conditional transition probability of transitioning to each
        # purview state, given the current mechanism state, so it needs to be of size
        # NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE PURVIEW STATES, but we start by marginalizing
        # over only the mechanism states, so NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE STATES
        mechanism_effect_repertoire = np.zeros((2**len(mechanism),2**self.num_nodes))
        
        # We marginalize over the mechanism states, which means we sum the probabilities of rows which
        # only differ in the state of nodes not in the mechanism.
        # We do this by finding the mechanism's state for a given row and mapping that row in the original tpm
        # to the correct row in the new mechanism repertoire
        for row in range(self.tpm.shape[0]):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            mechanism_effect_repertoire[self.state_to_index(mechanism_state),:] += self.tpm[row,:]
        
        # This is the final effect repertoire
        effect_repertoire = np.zeros((2**len(mechanism),2**len(purview)))
        
        # Second, we marginalize over the column states, which means we sum the probabilities of columns which
        # only differ in the state of nodes not in the purview.
        # We do this by finding the purview's state for a given column and mapping that row in the original tpm
        # to the correct column in the new effect repertoire
        for column in range(self.tpm.shape[1]):
            purview_state = self.index_to_state(column,self.num_nodes)[purview]
            effect_repertoire[:,self.state_to_index(purview_state)] += mechanism_effect_repertoire[:,column]
        
        # All that's left to do is normalize the rows because each row is a conditional probability distribution
        effect_repertoire = self.normalize_rows(effect_repertoire)
        
        # Now, we have to expand the effect_repertoire into the original state space which has all the 
        # possible current states at time t
        expanded_effect_repertoire = np.zeros((2**self.num_nodes,2**len(purview)))
        
        # This is done by mapping distributions in the effect_repertoire to each row in the expanded repertoire
        # where the mechanism's state matches
        for row in range(2**self.num_nodes):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            expanded_effect_repertoire[row,:] = effect_repertoire[self.state_to_index(mechanism_state),:]
            
        return expanded_effect_repertoire
    
    def effect_mip(self,system,purview):        
        # We need to find the cut the makes the least difference to the tpm of the system,
        # so we generate the original tpm first. We'll want to compare the original tpm
        # with the tpm generated after making our cut to determine the cut that makes the
        # least difference.
        min_cut = None
        cut_distance = float('inf')
        full_tpm = self.effect_repertoire(system,purview)
        
        # We need to loop through two powersets to determine all the cuts for this particular 
        # system + purview pair. We need to loop through the system powerset and the purview powerset
        # See factors.png below for an example.
        
        seen = set()
        
        for system_subset in self.powerset_all(system):
            
            for purview_subset in self.powerset_all(purview):
                
                # We haven't already seen the complement of the subset and the length of both of the factors
                # aren't zero
                if system_subset not in seen and not (len(system_subset) == 0 and len(purview_subset) == 0):
                    system_factor_1 = list(system_subset)
                    purview_factor_1 = list(purview_subset)
                    
                    system_factor_2 = list(np.setdiff1d(system,system_subset))
                    purview_factor_2 = list(np.setdiff1d(purview,purview_subset))
                    
                    tpm_1 = self.effect_repertoire(system_factor_1,purview_factor_1)
                    tpm_2 = self.effect_repertoire(system_factor_2,purview_factor_2)
                    
                    cut_tpm = self.tensor_product_ordered(tpm_1,purview_factor_1,tpm_2,purview_factor_2)
                    
                    # assess the distance between the cut_tpm and the full_tpm
                    new_cut_distance = stats.wasserstein_distance(full_tpm[self.state_to_index(self.states)],
                                                         cut_tpm[self.state_to_index(self.states)])
                    
#                     print ((system_factor_1,purview_factor_1,system_factor_2,purview_factor_2)," ",new_cut_distance)
                    
                    # update the distance with the lowest distance
                    if new_cut_distance <= cut_distance:
                        cut_distance = new_cut_distance
                        min_cut = (system_factor_1,purview_factor_1,system_factor_2,purview_factor_2)
            
            # add the complement of the system_subset to the seen set to ensure we don't double count certain
            # partitions
            seen.add(tuple(np.setdiff1d(system,system_subset)))
        
        return cut_distance,min_cut
    
    # TODO: implement this. Right now it's not working when the purview is smaller than the full
    # list of nodes because the tensor product assumes the second dimension will be the full length
    # of all the states given all the nodes
    def mie(self,system):
        phi = float('-inf')
        mie = None
        for purview in self.powerset_all(np.arange(self.num_nodes)):
            cut_distance,min_cut = self.effect_mip(system,list(purview))
            if cut_distance > phi:
                phi = cut_distance
                mie = purview
        return phi,mie
    
    def concept(self,system):
        
        effect_phi,mie = self.mie(system)
        cause_phi,mic = self.mic(system) # to be implemented
        
        return (min(effect_phi,cause_phi),mic,mie)
                        
    """    
    concept :
        input : list of nodes in system (in our case this will be the whole system)
        
        algorithm : min (mic(system),mie(system))
        
        output : phi value for that system
    """

    def cause_repertoire(self,purview,mechanism):
        # The cause repertoire captures the conditional transition probability of transitioning to each
        # purview state, given the current mechanism state, so it needs to be of size
        # NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE PURVIEW STATES, but we start by marginalizing
        # over only the mechanism states, so NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE STATES
        mechanism_cause_repertoire = np.zeros((2**len(mechanism),2**self.num_nodes))
        
        # We marginalize over the mechanism states, which means we sum the probabilities of rows which
        # only differ in the state of nodes not in the mechanism.
        # We do this by finding the mechanism's state for a given row and mapping that row in the original tpm
        # to the correct row in the new mechanism repertoire
        for row in range(self.tpm.shape[0]):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            mechanism_cause_repertoire[self.state_to_index(mechanism_state),:] += self.tpm[row,:]
        
        # This is the final cause repertoire
        cause_repertoire = np.zeros((2**len(mechanism),2**len(purview)))
        
        # Second, we marginalize over the column states, which means we sum the probabilities of columns which
        # only differ in the state of nodes not in the purview.
        # We do this by finding the purview's state for a given column and mapping that row in the original tpm
        # to the correct column in the new cause repertoire
        for column in range(self.tpm.shape[1]):
            purview_state = self.index_to_state(column,self.num_nodes)[purview]
            cause_repertoire[:,self.state_to_index(purview_state)] += mechanism_cause_repertoire[:,column]
        
        # All that's left to do is normalize the rows because each row is a conditional probability distribution
        cause_repertoire = self.normalize_rows(cause_repertoire)
        
        # Now, we have to expand the cause_repertoire into the original state space which has all the 
        # possible current states at time t
        expanded_cause_repertoire = np.zeros((2**self.num_nodes,2**len(purview)))
        
        # This is done by mapping distributions in the cause_repertoire to each row in the expanded repertoire
        # where the mechanism's state matches
        for row in range(2**self.num_nodes):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            expanded_cause_repertoire[row,:] = cause_repertoire[self.state_to_index(mechanism_state),:]
            
        # this ASSUMES there is only 1 thing in the purview, not sure what to do if there are more...
        # drop the column that are not active in purview
        # if len(purview) > 0:
        #     return expanded_cause_repertoire[:,self.states[purview[0]]]
        # else:
        #     return expanded_cause_repertoire
        return expanded_cause_repertoire
    
    def cause_tensor_product(self,t1,t2,purview):
        excluded_from_purview = set(range(len(starting_states))) - set(purview)
        # multiply possibilities of cause_repertoire and things not in cause_repertoire
        expanded_cause_repertoire_unsorted = np.array([])

        for i in range(len(t2)):
            expanded_cause_repertoire_unsorted = np.append(expanded_cause_repertoire_unsorted,t1 * t2[i])
        
        # make a copy of it
        expanded_cause_repertoire = np.zeros((len(expanded_cause_repertoire_unsorted)))

        # sort the copy
        ordered = list(purview) + list(excluded_from_purview)
        for i in range(len(expanded_cause_repertoire_unsorted)):
            # make binary array
            s = self.index_to_state(i,len(starting_states))
            #re-order digits
            reordered_s = np.full(len(s),9)
            for j in range(len(ordered)):
                reordered_s[j] = s[ordered[j]]
            index = self.state_to_index(reordered_s)
            expanded_cause_repertoire[i] = expanded_cause_repertoire_unsorted[index]
        return expanded_cause_repertoire

    def cause_mip(self,system,purview):        
        # We need to find the cut the makes the least difference to the tpm of the system,
        # so we generate the original tpm first. We'll want to compare the original tpm
        # with the tpm generated after making our cut to determine the cut that makes the
        # least difference.
        min_cut = None
        cut_distance = float('inf')
        full_tpm = self.cause_repertoire(system,purview)
        
        # We need to loop through two powersets to determine all the cuts for this particular 
        # system + purview pair. We need to loop through the system powerset and the purview powerset
        # See factors.png below for an example.
        
        seen = set()
        
        for system_subset in self.powerset_all(system):
            
            for purview_subset in self.powerset_all(purview):
                
                # We haven't already seen the complement of the subset and the length of both of the factors
                # aren't zero
                if system_subset not in seen and not (len(system_subset) == 0 and len(purview_subset) == 0):
                    system_factor_1 = list(system_subset)
                    purview_factor_1 = list(purview_subset)
                    
                    system_factor_2 = list(np.setdiff1d(system,system_subset))
                    purview_factor_2 = list(np.setdiff1d(purview,purview_subset))
                    
                    tpm_1 = self.cause_repertoire(system_factor_1,purview_factor_1)
                    tpm_2 = self.cause_repertoire(system_factor_2,purview_factor_2)
                    
                    cut_tpm = self.tensor_product_ordered(tpm_1,system_factor_1,tpm_2,system_factor_2)
                    # assess the distance between the cut_tpm and the full_tpm
                    new_cut_distance = stats.wasserstein_distance(full_tpm[self.state_to_index(self.states)],
                                                         cut_tpm[self.state_to_index(self.states)])
                    
#                     print ((system_factor_1,purview_factor_1,system_factor_2,purview_factor_2)," ",new_cut_distance)
                    
                    # update the distance with the lowest distance
                    if new_cut_distance <= cut_distance:
                        cut_distance = new_cut_distance
                        min_cut = (system_factor_1,purview_factor_1,system_factor_2,purview_factor_2)
            
            # add the complement of the system_subset to the seen set to ensure we don't double count certain
            # partitions
            seen.add(tuple(np.setdiff1d(system,system_subset)))
        
        return cut_distance,min_cut

    def mic(self,system):
        phi = float('-inf')
        mic = None
        for purview in self.powerset_all(np.arange(self.num_nodes)):
            cut_distance,min_cut = self.cause_mip(system,list(purview))
            if cut_distance >= phi:
                phi = cut_distance
                mic = purview
        return phi,mic
    
    def get_phi(self,system):
        phi = 0
        for s in all_subsets(system):
            p = self.concept(list(set(s)))
            if p[0] > phi and p[0] != float("inf"):
                phi = p[0]
        return phi
            
def all_subsets(ss):
    return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

In [84]:
#read in the files but exclude the row and column numbers
sleep = np.genfromtxt('sleep_mat/s4_sleep_t.csv', delimiter=',')[1:,1:]
wake = np.genfromtxt('sleep_mat/s4_wake_t.csv', delimiter=',')[1:,1:]

In [85]:
sleep.shape

(8, 8)

In [111]:
def index_to_state(index,num_states):
        binary = bin(index)[2:]
        states = []
        for state in binary: # convert row to binary
            states.append(int(state))
        
        states = np.flip(states) # flip to make little endian
       
        if (len(states) < num_states): 
            states = np.concatenate((states,np.zeros((num_states-len(states)),dtype=int)))
            
        return states

def pick_state(tpm1, tpm2):
    n = int(np.log2(tpm1.shape[0]))
    states_on = np.zeros((n))
    states_off = np.zeros((n))
    for i in range(tpm1.shape[0]):
        bi = index_to_state(i,n)
        total = np.sum(tpm1[:,i])
        for j in range(n):
            if bi[j] == 0:
                states_off[j] += total
            else:
                states_on[j] += total
    majority_states = []
    for i in range(n):
        if states_off[i] > states_on[i]:
            majority_states.append(0)
        else:
            majority_states.append(1)
    return majority_states

def pick_state2(tpm1, tpm2):
    n = int(np.log2(tpm1.shape[0]))
    max_i = 0
    max_s = 0
    for i in range(1,tpm1.shape[0]):
        s = np.sum(tpm1[:,i]) + np.sum(tpm2[:,i])
        if s > max_s:
            max_s = s
            max_i = i
    return index_to_state(max_i,n)


In [109]:
# pick a starting state - most common state across both TPMs
# is there a signficicance difference? between sleep and awake
# how does phi change on average as the number of nodes (channels) increases?


In [116]:
def get_phis(sleep_file,wake_file):
    sleep = np.genfromtxt(sleep_file, delimiter=',')[1:,1:]
    wake = np.genfromtxt(wake_file, delimiter=',')[1:,1:]
    starting_state = pick_state2(sleep,wake)
    print('starting states: ' + str(starting_state))
    sleep_phi = phi(sleep,starting_state)
    wake_phi = phi(wake,starting_state)
    sp = sleep_phi.get_phi([0,1,2])
    wp = wake_phi.get_phi([0,1,2])
    return sp,wp

def get_phis_for_state(sleep_file,wake_file,starting_state):
    sleep = np.genfromtxt(sleep_file, delimiter=',')[1:,1:]
    wake = np.genfromtxt(wake_file, delimiter=',')[1:,1:]
    print('starting states: ' + str(starting_state))
    sleep_phi = phi(sleep,starting_state)
    wake_phi = phi(wake,starting_state)
    sp = sleep_phi.get_phi([0,1,2])
    wp = wake_phi.get_phi([0,1,2])
    return sp,wp

def all_phis(sleep_list,wake_list,out_file_name):
    results = pd.DataFrame({'sleep':[],'awake':[]})
    results_count = 0
    for i in range(len(sleep_list)):
        sp,wp = get_phis(sleep_list[i],wake_list[i])
        results.loc[i] = [sp,wp]
    results.to_csv(out_file_name)
    
def all_phis_for_all_states(sleep_list,wake_list,out_file_name,n):
    results = pd.DataFrame({'sleep':[],'awake':[],'states':[]})
    results_count = 0
    results.to_csv(out_file_name)
    for j in range(n*n):
        state = index_to_state(j,n)
        for i in range(len(sleep_list)):
            sp,wp = get_phis_for_state(sleep_list[i],wake_list[i],state)
            results.loc[results_count] = [sp,wp,str(state)]
            results_count+=1
        results.to_csv(out_file_name, mode='a', header=False)
sl = ['sleep_mat/s3_sleep_t.csv','sleep_mat/s4_sleep_t.csv',
      'sleep_mat/s5_sleep_t.csv','sleep_mat/s6_sleep_t.csv',
      'sleep_mat/s7_sleep_t.csv','sleep_mat/s8_sleep_t.csv',
      'sleep_mat/s9_sleep_t.csv','sleep_mat/s10_sleep_t.csv',
      'sleep_mat/s11_sleep_t.csv','sleep_mat/s12_sleep_t.csv',
      'sleep_mat/s13_sleep_t.csv',
      'sleep_mat/s15_sleep_t.csv','sleep_mat/s16_sleep_t.csv',
      'sleep_mat/s17_sleep_t.csv','sleep_mat/s18_wake_t.csv',]
wl = ['sleep_mat/s3_wake_t.csv','sleep_mat/s4_wake_t.csv',
      'sleep_mat/s5_wake_t.csv','sleep_mat/s6_wake_t.csv',
      'sleep_mat/s7_wake_t.csv','sleep_mat/s8_wake_t.csv',
      'sleep_mat/s9_wake_t.csv','sleep_mat/s10_wake_t.csv',
      'sleep_mat/s11_wake_t.csv','sleep_mat/s12_wake_t.csv',
      'sleep_mat/s13_wake_t.csv',
      'sleep_mat/s15_wake_t.csv','sleep_mat/s16_wake_t.csv',
      'sleep_mat/s17_wake_t.csv','sleep_mat/s18_wake_t.csv',]
# all_phis(sl,wl,'nodes3-states2.csv')
all_phis_for_all_states(sl,wl,'nodes3-all-states.csv',3)
# print(get_phis('s3_sleep_t.csv','s3_wake_t.csv'))
# print(get_phis('sleep_mat/s4_sleep_t.csv','sleep_mat/s4_wake_t.csv'))

starting states: [0 0 0]


/Users/michael/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [0 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [1 0 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]
starting states: [0 1 0]


IndexError: index 8 is out of bounds for axis 0 with size 8